In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import anndata as ad
import os
import scanpy as sc

In [30]:
results = "../evaluation_results"
result_dfs = pd.concat({f.split("_results.txt")[0]: pd.read_csv(os.path.join(results, f)).set_index("test_group") for f in os.listdir(results) if "within" in f})

In [31]:
result_dfs.drop(["k", "reference"], axis=1)

p         z         s     n    m
                 test_group                                         
within_mcfarland 6           1.000000  0.124999  0.994898   388    4
                 6           0.652879  0.082793  0.994898   351   41
                 6           0.686497  0.272073  0.994898   320   72
                 6           0.419532 -0.358807  0.994898   263  129
                 6           0.784860  0.637119  0.994898   240  152
                 6           0.892735  1.094054  0.994898   209  183
                 24          0.127443 -2.386280  1.000000  1694   22
                 24          0.419192 -0.357334  1.000000  1549  167
                 24          0.713216  0.459330  1.000000  1400  316
                 24          0.959992  1.659447  1.000000  1194  522
                 24          0.824383  0.861295  1.000000  1010  706
                 24          0.181009 -0.979408  1.000000   898  818

In [32]:
result_dfs["n:m"] = result_dfs["n"].astype(str) + ":" + result_dfs["m"].astype(str)

In [33]:
result_dfs = result_dfs.drop(["n", "m", "s", "reference", "k", "z"], axis=1)

In [39]:
result_dfs

p       n:m
                 test_group                    
within_mcfarland 6           1.000000     388:4
                 6           0.652879    351:41
                 6           0.686497    320:72
                 6           0.419532   263:129
                 6           0.784860   240:152
                 6           0.892735   209:183
                 24          0.127443   1694:22
                 24          0.419192  1549:167
                 24          0.713216  1400:316
                 24          0.959992  1194:522
                 24          0.824383  1010:706
                 24          0.181009   898:818

In [40]:
sns.heatmap(result_dfs["p"])

IndexError: Inconsistent shape between the condition and the input (got (12, 1) and (12,))